In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

In [2]:
!unzip -f car_dataset.zip 

Archive:  car_dataset.zip


In [2]:
dataset = datasets.ImageFolder(
    'limit_dataset',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

In [3]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - 50, 50])

In [4]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

In [5]:
model = models.alexnet(pretrained=True)

In [6]:
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

In [7]:
device = torch.device('cuda')
model = model.to(device)

In [8]:
NUM_EPOCHS = 30
BEST_MODEL_PATH = 'limit_model.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):
    
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)-1
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
    
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)-0.2
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    print('%d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy

0: 0.200000
1: 0.200000
2: 0.200000
3: 0.200000
4: 0.200000
5: 0.200000
6: 0.200000
7: 0.200000
8: 0.200000
9: 0.200000
10: 0.200000
11: 0.200000
12: 0.200000
13: 0.200000
14: 0.200000
15: 0.200000
16: 0.200000
17: 0.200000
18: 0.200000
19: 0.200000
20: 0.200000
21: 0.200000
22: 0.200000
23: 0.200000
24: 0.200000
25: 0.200000
26: 0.200000
27: 0.200000
28: 0.200000
29: 0.200000
